# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [74]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [75]:
# Checking and showing current working directory
print(os.getcwd())

# Getting current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Creates a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# joins the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

#print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [76]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

# A csv file (event_database_new.csv) is created and added to the current working directory

In [77]:
# checks and displays the number of rows in the event_datafile_new.csv file

with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print('Lines: ', sum(1 for line in f))

Lines:  6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [78]:
# Creates and establishes connection to a Cassandra instance

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [79]:
# Creates a Keyspace 

try:   
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [80]:
# Set KEYSPACE to the keyspace specified above

try:  
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [88]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during |
## sessionId = 338, and itemInSession = 4

## Notice that the primary key is both sessionid and iteminsession fields. Both fields are needed in the|
## SELECT statement given the query. Remember that the query defines the data modeling when using NoSQL databases) 
query1 = "CREATE TABLE IF NOT EXISTS artist_song_length_table "
query1 = query1 + "(sessionid int, iteminsession int, artist text, song text, length float,\
                    PRIMARY KEY (sessionid, iteminsession))"
try:
  session.execute(query1)
except Exception as e:
    print(e)                  

In [89]:
# Assign filename to file, read it line by line We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_song_length_table (sessionid, iteminsession, artist, song, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        ## Notice that we need to convert length into a float and sessionid and iteminsession into integers |
        ## because in the csv file they are all strings
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [90]:
## Added the SELECT statement to verify the data was entered into the table and present the |
## artist, song title and song's length in the music app history that was heard during |
## sessionId = 338, and itemInSession = 4 as requested

query= "SELECT artist, song, length \
        FROM artist_song_length_table \
        WHERE sessionid = 338 AND iteminsession = 4"
try:
   rows = session.execute(query)
except Exception as e:
   print(e)
for row in rows:
    print ('[Artist]: '+ row.artist, '[Song Title]: '+row.song, '[Length]:', row.length )

[Artist]: Faithless [Song Title]: Music Matters (Mark Knight Dub) [Length]: 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [91]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

## Notice that partition key is (userid, sessionid) while iteminsession is used a clustering column |
## that will provide the sorting functionality

query2 = "CREATE TABLE IF NOT EXISTS artist_song_user_table "
query2 = query2 + "(userid int, sessionid int, iteminsession int, \
                    artist text, song text, firstname text, lastname text, \
                    PRIMARY KEY ((userid, sessionid), iteminsession))"
try:
  session.execute(query2)
except Exception as e:
    print(e)   
    
# Assign filename to file, read it line by line and insert designated fields into the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_song_user_table (userid, sessionid, iteminsession, artist, song, firstname, lastname) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Notice that we need to convert fields into the right format based on the table definition |
        ## because in the csv file they are all strings
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        
## Added the SELECT statement to verify the data was entered into the table correctly and present name of artist, |
## song title (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

query= "SELECT artist, song, firstname, lastname \
        FROM artist_song_user_table \
        WHERE userid = 10 AND sessionid = 182"
try:
   rows = session.execute(query)
except Exception as e:
   print(e)
for row in rows:
    print ('[Artist]: '+ row.artist, '[Song Title]: '+ row.song, '[User]: '+ row.firstname, row.lastname)        

[Artist]: Down To The Bone [Song Title]: Keep On Keepin' On [User]: Sylvie Cruz
[Artist]: Three Drives [Song Title]: Greece 2000 [User]: Sylvie Cruz
[Artist]: Sebastien Tellier [Song Title]: Kilometer [User]: Sylvie Cruz
[Artist]: Lonnie Gordon [Song Title]: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) [User]: Sylvie Cruz


In [92]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## Notice that partition key is userid while userdid is used a clustering column |
## that will provide the sorting functionality
query3 = "CREATE TABLE IF NOT EXISTS users_songs_table "
query3 = query3 + "(song text, userid int, firstname text, lastname text, \
                    PRIMARY KEY ((song), userid))"
try:
  session.execute(query3)
except Exception as e:
    print(e)   
    
# Assign filename to file, read it line by line and insert designated fields into the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO users_songs_table (song, userid, firstname, lastname) "
        query = query + "VALUES (%s, %s, %s, %s)"
        ## Notice that we need to convert fields into the right format based on the table definition |
        ## because in the csv file they are all strings
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        
## Added the SELECT statement to verify the data was entered into the table correctly and present |
## user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query= "SELECT firstname, lastname \
        FROM users_songs_table \
        WHERE song = 'All Hands Against His Own'"
try:
   rows = session.execute(query)
except Exception as e:
   print(e)
for row in rows:
    print ('[User]: '+ row.firstname, row.lastname)                            

[User]: Jacqueline Lynch
[User]: Tegan Levine
[User]: Sara Johnson


### Drop the tables before closing out the sessions

In [20]:
## Drop the table before closing out the session

In [93]:
query= "DROP TABLE artist_song_length_table"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query= "DROP TABLE artist_song_user_table "
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query= "DROP TABLE users_songs_table "
try:
    session.execute(query)
except Exception as e:
    print(e)    

### Close the session and cluster connection¶

In [94]:
session.shutdown()
cluster.shutdown()